In [ ]:
import xarray
from dask.distributed import Client
from glob import glob


In [ ]:
phts = [pth for pth in glob('/snx300/scratch/kschuurm/DATA/customized/HRSEVIRI_201*')]
dss = [ xarray.open_dataset(pth, engine='h5netcdf').chunk({'time':60, 'lat':-1, 'lon':-1}) for pth in phts]
hres = xarray.concat(dss, dim='time', data_vars='minimal', compat='equals', coords= 'minimal')

hres = hres.sel(lat=slice(None, 62))



In [ ]:
sis = xarray.open_mfdataset('SIS_*.nc', concat_dim='time', combine='nested', data_vars='minimal', coords='minimal', compat='override', engine='h5netcdf')

intersec_tidx = sorted(
        list(
            set(hres.time.dt.round("30min").values).intersection(
                set(sis.time.dt.round("30min").values)
            )
        )
    )

hres_tidx_sort = hres.time.copy().sortby(hres.time)
hres_intersec_tidx = hres_tidx_sort.sel(time=intersec_tidx, method='nearest')

sis_reindex = sis.reindex(time=hres_intersec_tidx, lat=hres.lat, lon=hres.lon, method='nearest')
shape = hres.channel_1.shape
combined = xarray.merge([hres, sis_reindex]).drop('crs')

encoding = {}
for key in combined.variables:
    if key == 'record_status':
        encoding[key] = {'chunks': 60}
    else:
        encoding[key] = {'chunks':(60, shape[1], shape[2])}

In [ ]:
hres.to_zarr('HRSEVIRI.zarr', mode='w', consolidated=True) # consolidate means storing metadata in one file instead of seperate. Needed because of file limit.
